In [5]:
import os
import json
import gzip
import io
import glob
import pickle

In [6]:
#Saving all the dates in date_dict

if(os.path.exists('/dates_dict.pkl')):
    pkl_file = open('/dates_dict.pkl', 'rb')
    date_dict = pickle.load(pkl_file)
    pkl_file.close()

else: date_dict = {}
    
all_gz_files = glob.glob("/dnc2020/**/*.gz")

for file_path in all_gz_files:
    path_arr = file_path.split("/")
    date = path_arr[4].split()
    if date[0] not in date_dict:
        date_dict[date[0]] = "not_processed"

output = open('/dates_dict.pkl', 'wb')
pickle.dump(date_dict, output)
output.close()


if(os.path.exists('/tweets_count.pkl')):
    pkl_file = open('/tweets_count.pkl', 'rb')
    tweets_count = pickle.load(pkl_file)
    pkl_file.close()

else: tweets_count = {}

# date_dict

In [7]:
#function to extract the tweet text form the raw json of a tweet

def getTweetText(tweet):
    temp_text = ""
    if "extended_tweet" in tweet:
        tweet_details = tweet["extended_tweet"]
        temp_text += "" +tweet_details["full_text"]
    
    #if any tweet was retweeted
    elif "retweeted_status" in tweet:
        #if extended text available in the original tweet
        if("extended_tweet" in tweet["retweeted_status"]):
            tweet_details = tweet["retweeted_status"]["extended_tweet"]
            temp_text += "" +tweet_details["full_text"]
                    
        #getting the text of the original
        elif "text" in tweet["retweeted_status"]:
            tweet_details = tweet["retweeted_status"]["text"]
            temp_text += "" +tweet_details
    else:
        temp_text += "" +tweet["text"]
            
    return temp_text

In [8]:

for date in date_dict:
    if date_dict[date] == "not_processed":
        all_gz_files = glob.glob("/dnc2020/" +date+"*/*.gz")    
        tweets = {}  #dict
        
        i=0 #keeps the track of # of files for a date
        j=0 #keeps a track of the # of tweets in a file
        total_tweets = 0 #keeps a track of the total tweets on a date
        for file_path in all_gz_files:
            i +=1; 
            j = 0 
            print(str(i)+"/"+str(len(all_gz_files))+"...reading file "+file_path.rstrip())
            try:
                gz = gzip.open(file_path, 'r')
                f = io.BufferedReader(gz)

                #creating a dictionary of the tweets [(tweet_id)key : (tweet)value]
                for line in f:
                    j +=1
                    if(j%100==0):
                        print("..........processed " +str(j) +" tweets", end="\r" )
                    total_tweets +=1
#                     if(j%1000000==0): break   #limiting to reading 1M tweets per file
                    temp_tweet_json = json.loads(line)
                    key = temp_tweet_json["id"]
                    
                    #creating a dict of only the essential fields from the tweet to save in the 'tweets' dict
                    value = {}
                    value['created_at'] = temp_tweet_json['created_at']
                    value['tweet_text'] = getTweetText(temp_tweet_json)
                    value['id'] = temp_tweet_json["id"]
                    value['user_id'] = temp_tweet_json['user']['id']
                    if "retweeted_status" in temp_tweet_json:
                        value['hashtags'] = temp_tweet_json['retweeted_status']['entities']['hashtags']
                        value['user_mentions'] = temp_tweet_json['retweeted_status']['entities']['user_mentions']
                    else: 
                        value['hashtags'] = temp_tweet_json['entities']['hashtags']
                        value['user_mentions'] = temp_tweet_json['entities']['user_mentions']
                        
                    tweets.update( {key : value} )
                print("..........processed " +str(j) +" tweets")
             
            except EOFError as error:
                error_log_file = open("EOFError_files.txt","a+") 
                error_log_file.write(file_path + "|" + str(total_tweets) + "\n");                  
                error_log_file.close()
                print("EOFError")

        print()
        print("Number of tweets on " + date +": " +str(total_tweets))
        print()
        
        output = open('/Processed Tweets/tweets_' +date+'.pkl', 'wb')
        pickle.dump(tweets, output)
        output.close()
              
        date_dict[date] = 'processed'
        tweets_count[date] = total_tweets
             
        output = open('/dates_dict.pkl', 'wb')
        pickle.dump(date_dict, output)
        output.close()
        
        output = open('/tweets_count.pkl', 'wb')
        pickle.dump(tweets_count, output)
        output.close()



1/17...reading file /dnc2020/2020-03-26 13:50:34.030000--4/output_4.txt.gz
..........processed 29766 tweets
2/17...reading file /dnc2020/2020-03-26 15:26:10.532562--1/output_1.txt.gz
EOFError..processed 7800 tweets
3/17...reading file /dnc2020/2020-03-26 13:11:57.916117--2/output_2.txt.gz
..........processed 1113 tweets
4/17...reading file /dnc2020/2020-03-26 16:56:59.179055--4/output_4.txt.gz
EOFError..processed 563900 tweets
5/17...reading file /dnc2020/2020-03-26 16:56:59.180648--5/output_5.txt.gz
..........processed 89364 tweets
6/17...reading file /dnc2020/2020-03-26 13:28:30.686000--1/output_1.txt.gz
EOFError..processed 8400 tweets
7/17...reading file /dnc2020/2020-03-26 14:50:52.541987--4/output_4.txt.gz
EOFError..processed 18300 tweets
8/17...reading file /dnc2020/2020-03-26 15:26:10.532920--2/output_2.txt.gz
EOFError..processed 19700 tweets
9/17...reading file /dnc2020/2020-03-26 13:23:00.597409--5/output_5.txt.gz
EOFError..processed 21900 tweets
10/17...reading file /dnc2020/